In [ ]:
### Fixing import errors of the

import sys
import os

# This code navigates up one directory from the notebook's location ('examples/')
# to get the project's root directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# We check if the path is already in the system path.
# If not, we add it to the beginning of the list.
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to Python path: {project_root}")
else:
    print(f"Project root is already in Python path: {project_root}")

# Optional: You can print the first few paths to verify
print("\nVerifying sys.path:")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")

In [ ]:
import json
from llama_index.core import Document
from src.GlirelPathExtractor import GlirelPathExtractor 
from src.RecursivePathExtractor import RecursiveLLMPathExtractor
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex,Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
import nest_asyncio
import openlit


In [ ]:
openlit.init(
  otlp_endpoint="http://127.0.0.1:4318",
  application_name="llm",
  environment="obama_enviroment"
  
)

In [ ]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [ ]:
with open('../.data/novel.json', 'r') as file:
    # Load the JSON data from the file into a Python object
    data = json.load(file)

In [ ]:
print(data[0]["corpus_name"])

In [ ]:
documents = [Document(text=t["context"],id_=t["corpus_name"]) for t in data]

In [ ]:
doc = documents[1]
doc.id_

In [ ]:
extractor = RecursiveLLMPathExtractor(llm=Settings.llm,
    num_workers=1,
    max_paths_per_chunk=15)


In [ ]:
# for starting later
#documents = documents[2:]
documents[0].id_

In [ ]:
nest_asyncio.apply()
for document in documents:
    index = PropertyGraphIndex.from_documents(
        documents=[document],
        kg_extractors=[extractor],
        use_async = False,   
    )
    novel_id = document.id_
    index.storage_context.persist(persist_dir=f"./.persistent_storage/llm/{novel_id}")
